<a href="https://colab.research.google.com/github/nicholashaddou/NNDL-Final-Project/blob/main/NewDL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets evaluate seqeval --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you hav

In [ ]:
# Import libraries
import re
from collections import defaultdict
from transformers import BertConfig, BertForTokenClassification
import evaluate
import numpy as np
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch
from sklearn.model_selection import train_test_split
from datasets import Dataset

Prepare DataSet

In [ ]:
# Function to parse PubTator file
def parse_pubtator_file(filepath):
    data = []
    annotations = defaultdict(list)
    with open(filepath, 'r') as f:
        lines = f.readlines()

    current_text = ""
    current_pm_id = ""

    for line in lines:
        line = line.strip()
        if not line:
            continue

        if "|t|" in line or "|a|" in line:
            parts = line.split("|")
            pmid = parts[0]
            text = parts[2]
            if "|t|" in line:
                current_text = text
                current_pm_id = pmid
            else:
                current_text += " " + text
        else:
            parts = line.split("\t")
            if len(parts) >= 6:
                start, end, mention, entity_type, entity_id = int(parts[1]), int(parts[2]), parts[3], parts[4], parts[5]
                entity_type = entity_type.upper()
                annotations[current_pm_id].append((start, end, mention, entity_type))

        if line == lines[-1] or "|a|" in line:
            if current_pm_id and current_text:
                data.append({
                    "pmid": current_pm_id,
                    "text": current_text,
                    "annotations": annotations[current_pm_id]
                })
    return data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# After drive is mounted
data_path_train = "/content/drive/MyDrive/DL/CDR_Data/CDR.Corpus.v010516/CDR_TrainingSet.PubTator.txt"
data_path_dev = "/content/drive/MyDrive/DL/CDR_Data/CDR.Corpus.v010516/CDR_DevelopmentSet.PubTator.txt"

# Parse both files
parsed_train = parse_pubtator_file(data_path_train)
parsed_dev = parse_pubtator_file(data_path_dev)

# Combine them
full_parsed_data = parsed_train + parsed_dev

Load DataSet

In [ ]:
# Load your dataset
#data_path = '/content/drive/MyDrive/DL/CDR_Data/CDR.Corpus.v010516/CDR_TrainingSet.PubTator.txt'  # Update to your correct path
#dataset = parse_pubtator_file(data_path)

Build tokens and BIO labels

In [ ]:
# Build tokens and BIO labels
label_list = ["O", "B-CHEMICAL", "I-CHEMICAL", "B-DISEASE", "I-DISEASE"]
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for label, i in label2id.items()}

def prepare_ner_dataset(parsed_data):
    tokenized_texts = []
    labels = []

    for item in parsed_data:
        text = item['text']
        entities = item['annotations']
        tokens = text.split()
        label_seq = ['O'] * len(tokens)

        for start, end, mention, entity_type in entities:
            entity_type = entity_type.upper()
            for idx, token in enumerate(tokens):
                if mention in token:
                    label_seq[idx] = "B-" + entity_type if label_seq[idx] == 'O' else "I-" + entity_type

        tokenized_texts.append(tokens)
        labels.append([label2id[label] for label in label_seq])

    return tokenized_texts, labels

tokens, ner_labels = prepare_ner_dataset(full_parsed_data)

from transformers import AutoTokenizer
from datasets import Dataset

# Load tokenizer (same for all models)
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# Label mappings
label_list = ["O", "B-CHEMICAL", "I-CHEMICAL", "B-DISEASE", "I-DISEASE"]
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for label, i in label2id.items()}

# Wrap in Hugging Face Dataset
train_dataset = Dataset.from_dict({
    'tokens': tokens,
    'ner_tags': ner_labels
})

# Tokenize + align labels (using your existing function)
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        padding="max_length",  # ensure uniform length
        is_split_into_words=True
    )

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx])
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs
train_dataset = train_dataset.map(tokenize_and_align_labels, batched=True)

from datasets import DatasetDict

# 90% train, 10% eval
dataset_split = train_dataset.train_test_split(test_size=0.1)
train_ds = dataset_split["train"]
eval_ds = dataset_split["test"]

print("Example tokens:", tokens[0])
print("Example labels:", ner_labels[0])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Example tokens: ['Naloxone', 'reverses', 'the', 'antihypertensive', 'effect', 'of', 'clonidine.', 'In', 'unanesthetized,', 'spontaneously', 'hypertensive', 'rats', 'the', 'decrease', 'in', 'blood', 'pressure', 'and', 'heart', 'rate', 'produced', 'by', 'intravenous', 'clonidine,', '5', 'to', '20', 'micrograms/kg,', 'was', 'inhibited', 'or', 'reversed', 'by', 'nalozone,', '0.2', 'to', '2', 'mg/kg.', 'The', 'hypotensive', 'effect', 'of', '100', 'mg/kg', 'alpha-methyldopa', 'was', 'also', 'partially', 'reversed', 'by', 'naloxone.', 'Naloxone', 'alone', 'did', 'not', 'affect', 'either', 'blood', 'pressure', 'or', 'heart', 'rate.', 'In', 'brain', 'membranes', 'from', 'spontaneously', 'hypertensive', 'rats', 'clonidine,', '10(-8)', 'to', '10(-5)', 'M,', 'did', 'not', 'influence', 'stereoselective', 'binding', 'of', '[3H]-naloxone', '(8', 'nM),', 'and', 'naloxone,', '10(-8)', 'to', '10(-4)', 'M,', 'did', 'not', 'influence', 'clonidine-suppressible', 'binding', 'of', '[3H]-dihydroergocryptine',



Scratch BERT model



In [ ]:
# Define BERT config from scratch
config = BertConfig(
    vocab_size=tokenizer.vocab_size,
    num_labels=len(label_list),
    hidden_size=768,
    num_hidden_layers=12,
    num_attention_heads=12,
    intermediate_size=3072,
    id2label=id2label,
    label2id=label2id
)

# Initialize model with random weights (from scratch)
model = BertForTokenClassification(config)

Scratch BERT Evaluation metric

In [ ]:
# Load seqeval metric
metric = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = [[id2label[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [id2label[pred] for pred, lab in zip(prediction, label) if lab != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Scratch BERT Train using trainer

In [ ]:
from transformers import BertConfig, BertForTokenClassification, TrainingArguments, Trainer

# 🧠 Define a fresh (untrained) BERT model
config = BertConfig(
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id
)
model_scratch = BertForTokenClassification(config)

# ✅ Training arguments
training_args = TrainingArguments(
    output_dir="./results_scratch",
    learning_rate=1e-5,
    per_device_train_batch_size=4,
    num_train_epochs=30,
    weight_decay=0.01,
    logging_dir='./logs_scratch',
    logging_steps=10,
    report_to="none"
)

# ✅ Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# 🚀 Train and evaluate
trainer.train()
results = trainer.evaluate()
print(results)


<ipython-input-9-b1d718a01528>:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
10,0.945500
20,0.797700
30,0.761400
40,0.718800
50,0.699900
60,0.686700
70,0.822600
80,0.694000
90,0.656700
100,0.699700


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained("/content/drive/MyDrive/DL/saved_scratch_ner")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/DL/saved_scratch_ner")

model_scratch.save_pretrained("/content/drive/MyDrive/DL/saved_scratch_ner")
tokenizer.save_pretrained("/content/drive/MyDrive/DL/saved_scratch_ner")

Prediction of Scratch BERT model

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

# Load model and tokenizer
model_path = "/content/drive/MyDrive/DL/saved_scratch_ner"
tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)
model = AutoModelForTokenClassification.from_pretrained(model_path, local_files_only=True)
model.eval()

def predict_entities(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)

    predictions = torch.argmax(outputs.logits, dim=2)[0].tolist()
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    labels = [model.config.id2label[p] for p in predictions]

    print("\n🔎 Clean Entity Prediction:")
    for token, label in zip(tokens, labels):
        if token in ["[CLS]", "[SEP]"]:
            continue
        if label != "O":
            # Clean subwords like '##tam' → 'tam'
            clean_token = token.replace("##", "")
            print(f"{clean_token}: {label}")

# Run prediction
predict_entities(
    "Paracetamol helps reduce fever and is often used during COVID-19.",
    model,
    tokenizer
)


🔎 Clean Entity Prediction:
Para: B-DISEASE
ce: B-CHEMICAL
tam: B-CHEMICAL
helps: B-DISEASE
reduce: B-DISEASE
fever: I-DISEASE
and: I-DISEASE
is: B-CHEMICAL
used: B-CHEMICAL
during: B-CHEMICAL
CO: I-DISEASE
VI: B-CHEMICAL
D: B-CHEMICAL
-: B-DISEASE
.: B-CHEMICAL


In [ ]:
!ls /content/drive/MyDrive/DL/saved_scratch_ner

ls: cannot access '/content/drive/MyDrive/DL/saved_scratch_ner': No such file or directory


Mid Level: BERT-base-cased

In [ ]:

model_mid = AutoModelForTokenClassification.from_pretrained(
    "bert-base-cased",
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id
)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Define training arguments

In [ ]:

training_args_mid = TrainingArguments(
    output_dir="./results_mid",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs_mid",
    logging_steps=10,
    report_to="none"
)

Define Trainer and Train

In [ ]:

trainer_mid = Trainer(
    model=model_mid,
    args=training_args_mid,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer_mid.train()
results_mid = trainer_mid.evaluate()
print(results_mid)

<ipython-input-15-f9e3d1d7d761>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_mid = Trainer(


Step,Training Loss
10,0.769000
20,0.425600
30,0.324800
40,0.321500
50,0.271400
60,0.260100
70,0.202600
80,0.197200
90,0.178800
100,0.135500


{'eval_loss': 0.19840151071548462, 'eval_precision': 0.6541095890410958, 'eval_recall': 0.6779059449866903, 'eval_f1': 0.6657952069716775, 'eval_accuracy': 0.9313441278370925, 'eval_runtime': 1.6889, 'eval_samples_per_second': 29.604, 'eval_steps_per_second': 4.145, 'epoch': 3.0}


In [ ]:
model_mid.save_pretrained("./saved_mid_ner")
tokenizer.save_pretrained("./saved_mid_ner")

('./saved_mid_ner/tokenizer_config.json',
 './saved_mid_ner/special_tokens_map.json',
 './saved_mid_ner/vocab.txt',
 './saved_mid_ner/added_tokens.json',
 './saved_mid_ner/tokenizer.json')

Prediction mid BERT-case-based

In [ ]:


model_path = "./saved_mid_ner"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForTokenClassification.from_pretrained(model_path)

model.eval()

def predict_entities(text, model, tokenizer):
    tokens = tokenizer(text, return_tensors="pt", truncation=True)
    with torch.no_grad():
        outputs = model(**tokens)
    predictions = torch.argmax(outputs.logits, dim=2)
    input_tokens = tokenizer.convert_ids_to_tokens(tokens["input_ids"][0])
    predicted_labels = [model.config.id2label[p.item()] for p in predictions[0]]

    print("\n🔎 Entity Prediction:")
    for token, label in zip(input_tokens, predicted_labels):
        if label != "O":
            print(f"{token}: {label}")

predict_entities(
    "Paracetamol helps reduce fever and is often used during COVID-19.",
    model,
    tokenizer
)


🔎 Entity Prediction:
Para: B-CHEMICAL
##ce: B-CHEMICAL
##tam: B-CHEMICAL
##ol: B-CHEMICAL
fever: B-DISEASE


High Level: BioBERT Eval(Evaluation) dataset

In [ ]:

# ✅ Split into 80% train and 20% eval
train_tokens, eval_tokens, train_labels, eval_labels = train_test_split(
    tokens, ner_labels, test_size=0.2, random_state=42
)

# ✅ Create Hugging Face Dataset objects
train_dataset = Dataset.from_dict({
    'tokens': train_tokens,
    'ner_tags': train_labels
})

eval_dataset = Dataset.from_dict({
    'tokens': eval_tokens,
    'ner_tags': eval_labels
})

In [ ]:


# Load BioBERT tokenizer
model_name = "dmis-lab/biobert-base-cased-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Create Hugging Face Dataset objects using train-test split
train_dataset = Dataset.from_dict({
    'tokens': train_tokens,
    'ner_tags': train_labels
})

eval_dataset = Dataset.from_dict({
    'tokens': eval_tokens,
    'ner_tags': eval_labels
})

# Tokenize and align labels
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples['tokens'],
        padding="max_length",
        truncation=True,
        max_length=512,
        is_split_into_words=True,
        return_tensors=None
    )
    labels = []

    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx])
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Apply tokenizer to both train and eval datasets
train_dataset = train_dataset.map(tokenize_and_align_labels, batched=True)
eval_dataset = eval_dataset.map(tokenize_and_align_labels, batched=True)

config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Load Model and Set Trainer

In [ ]:


# Load BioBERT model
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id
)

# Evaluation metric
metric = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_labels = [[label_list[l] for l in label if l != -100] for label in labels]
    true_preds = [[label_list[p] for (p, l) in zip(pred, label) if l != -100] for pred, label in zip(predictions, labels)]
    results = metric.compute(predictions=true_preds, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"]
    }

# Training setup
training_args = TrainingArguments(
    output_dir="./results_biobert",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs_biobert",
    report_to="none"
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Train and evaluate
trainer.train()
results = trainer.evaluate()
print(results)

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

<ipython-input-20-7d7d22f9f732>:38: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


{'eval_loss': 0.19568979740142822, 'eval_precision': 0.6387270282384581, 'eval_recall': 0.5461862782675354, 'eval_f1': 0.5888429752066116, 'eval_accuracy': 0.9308442671486895, 'eval_runtime': 4.3485, 'eval_samples_per_second': 22.996, 'eval_steps_per_second': 2.99, 'epoch': 3.0}


In [ ]:

model.save_pretrained("./saved_biobert_ner")
tokenizer.save_pretrained("./saved_biobert_ner")

('./saved_biobert_ner/tokenizer_config.json',
 './saved_biobert_ner/special_tokens_map.json',
 './saved_biobert_ner/vocab.txt',
 './saved_biobert_ner/added_tokens.json',
 './saved_biobert_ner/tokenizer.json')

Prediction high level bioBERT

In [ ]:


model_path = "./saved_biobert_ner"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForTokenClassification.from_pretrained(model_path)

model.eval()

def predict_entities(text, model, tokenizer):
    tokens = tokenizer(text, return_tensors="pt", truncation=True)
    with torch.no_grad():
        outputs = model(**tokens)
    predictions = torch.argmax(outputs.logits, dim=2)
    input_tokens = tokenizer.convert_ids_to_tokens(tokens["input_ids"][0])
    predicted_labels = [model.config.id2label[p.item()] for p in predictions[0]]

    print("\n🔎 Entity Prediction:")
    for token, label in zip(input_tokens, predicted_labels):
        if label != "O":
            print(f"{token}: {label}")

predict_entities(
    "Paracetamol helps reduce fever and is often used during COVID-19.",
    model,
    tokenizer
)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



🔎 Entity Prediction:
para: B-CHEMICAL
##ce: B-CHEMICAL
##tam: B-CHEMICAL
##ol: B-CHEMICAL


In [ ]:

model = AutoModelForTokenClassification.from_pretrained("./saved_biobert_ner")
tokenizer = AutoTokenizer.from_pretrained("./saved_biobert_ner")

In [ ]:
!cp -r /content/saved_biobert_ner /content/drive/MyDrive/

In [ ]:
!ls /content/drive/MyDrive/saved_biobert_ner


config.json	   special_tokens_map.json  tokenizer.json
model.safetensors  tokenizer_config.json    vocab.txt


Prediction

Streamlit App

In [ ]:
!zip -r saved_biobert_ner.zip saved_biobert_ner

  adding: saved_biobert_ner/ (stored 0%)
  adding: saved_biobert_ner/tokenizer.json (deflated 70%)
  adding: saved_biobert_ner/model.safetensors (deflated 7%)
  adding: saved_biobert_ner/vocab.txt (deflated 49%)
  adding: saved_biobert_ner/tokenizer_config.json (deflated 74%)
  adding: saved_biobert_ner/config.json (deflated 52%)
  adding: saved_biobert_ner/special_tokens_map.json (deflated 42%)


In [ ]:
from google.colab import files
files.download('saved_biobert_ner.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>